# core

> Fill in a module description here

In [1]:
#| default_exp drive

In [10]:
#| exporti

from nbdev.showdoc import *
from enum import Enum
import datetime as dt
from dataclasses import dataclass

from googleapiclient.discovery import build



In [3]:
#| export

class GoogleDrive_MimeType_Enum(Enum):
    """for translating Google Drive types into 'exports as'"""
    
    docx  = 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
    pptx = 'application/vnd.openxmlformats-officedocument.presentationml.presentation'
    
    odt  = 'application/vnd.oasis.opendocument.text'
    rtf = 'application/rtf'
    pdf = 'application/pdf'
    txt = 'text/plain'
    # html = 'application/zip'
    zip = 'application/zip'
    epub  = 'application/epub+zip'
    
    folder = 'application/vnd.google-apps.folder'
    gdoc = 'application/vnd.google-apps.document'   
    gsheet ='application/vnd.google-apps.spreadsheet'
    gshortcut = 'application/vnd.google-apps.shortcut'
    gslides = 'application/vnd.google-apps.presentation'
    
    # 'tex'  = 'application/zip'
    # 'html.zip'= 'application/zip'

In [4]:
for member in GoogleDrive_MimeType_Enum:
    print(member.name, member.value)

docx application/vnd.openxmlformats-officedocument.wordprocessingml.document
pptx application/vnd.openxmlformats-officedocument.presentationml.presentation
odt application/vnd.oasis.opendocument.text
rtf application/rtf
pdf application/pdf
txt text/plain
zip application/zip
epub application/epub+zip
folder application/vnd.google-apps.folder
gdoc application/vnd.google-apps.document
gsheet application/vnd.google-apps.spreadsheet
gshortcut application/vnd.google-apps.shortcut
gslides application/vnd.google-apps.presentation


# Google Drive File Looper
retrieves a listing of all the files and subfiles in a folder

In [21]:
@dataclass
class gdoc_file:
   mime_type : str
   webViewLink : str
   doc_id  : str
   doc_name : str
   modified_time : dt.datetime 
   folder_path : str
   doc_type : str

   @classmethod
   def _from_json(cls, obj :dict):
      cls( mime_type = obj['mimeType'],
      webViewLink = obj['webViewLink'],
      doc_id = obj['id'],
      doc_name = obj['name'],
modified_time = ']
  'type': 'gslides'}
      


In [5]:
#| exporti

def process_file(file_obj, folder_path):
    """gets the file type using GoogleDrive Enum, useful for processing & downloading a file"""

    file_obj.update({'folder_path': folder_path})

    file_type = "undefined_enum"
    try:
        file_type = GoogleDrive_MimeType_Enum(file_obj['mimeType']).name
    
    except Exception as e:
        print(e)
    
    finally:
        file_obj.update({'type': file_type})
        return file_obj


def get_files(folder_id, service):
    """retrieves files in a folder"""

    page_token = None
    file_ls = []

    while True:
        res = service.files().list(
            q=f"'{folder_id}' in parents",
            pageSize=10, fields="nextPageToken, files(id,webViewLink, name, mimeType,modifiedTime )",
            pageToken=page_token).execute()

        items = res.get('files', [])

        file_ls += items

        page_token = res.get('nextPageToken', None)

        if page_token is None:
            break

    return file_ls


def get_file_looper(folder_id, service, file_ls=None, folder_path=''):
    """recursive function to get files in a folder and map over files in subfolder"""

    if not file_ls:
        file_ls = []
        
    new_files = get_files(folder_id=folder_id, service=service)

    new_files = [process_file(file_obj, folder_path)for file_obj in new_files]

    file_ls += new_files

    if new_files:
        for file_obj in new_files:
            if file_obj['type'] == 'folder':

                sub_folder_path = os.path.join(folder_path, clean_url_name(convert_to_snake(file_obj.get('name'))))

                get_file_looper(folder_id=file_obj.get('id'),
                                folder_path=sub_folder_path,
                                service=service,
                                file_ls=file_ls)

    return file_ls




In [6]:
def get_folder_contents(folder_id, creds, folder_path='') -> [dict] # returns list of file objects
:
    """creates a list files in a folder using a set of credentials"""

    service = build(serviceName='drive',
                    version='v3',
                    credentials=creds)

    return get_file_looper(folder_id=folder_id, service=service, folder_path=folder_path)


In [9]:
from dotenv import load_dotenv
import json
import os
import gdoc_sync.creds as gcred

FOLDER_ID ='1SRrD1dNgZgHYjnhkJtARbLhydMP94qWi'
# loads client secret information from .env file into env
load_dotenv('.env')

# loads client secret information into dict
client_secret_info=json.loads(os.environ['GDOC_KEY'])
token_info = json.loads(os.environ['GDOC_TOKEN'])

# generates Credentials object
creds = gcred.generate_creds_object(client_secret_info=client_secret_info, token_info = token_info)

get_folder_contents(folder_id = FOLDER_ID, creds = creds, folder_path='test')

[{'mimeType': 'application/vnd.google-apps.presentation',
  'webViewLink': 'https://docs.google.com/presentation/d/1_k4NRraKI1TmHNlpQCuqJrWr6dP7DNracdMCtfN8XlM/edit?usp=drivesdk',
  'id': '1_k4NRraKI1TmHNlpQCuqJrWr6dP7DNracdMCtfN8XlM',
  'name': 'sample slide',
  'modifiedTime': '2023-08-17T13:54:31.973Z',
  'folder_path': 'test',
  'type': 'gslides'},
 {'mimeType': 'application/vnd.google-apps.document',
  'webViewLink': 'https://docs.google.com/document/d/1j7XsbvFy0xUgGL6i-3LSChKvzSmTZSOyimEt6tQS-Kk/edit?usp=drivesdk',
  'id': '1j7XsbvFy0xUgGL6i-3LSChKvzSmTZSOyimEt6tQS-Kk',
  'name': 'Sample Doc for Google_Sync project',
  'modifiedTime': '2023-08-17T13:53:01.574Z',
  'folder_path': 'test',
  'type': 'gdoc'}]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()